In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
import sklearn.datasets as datasets

In [9]:
cancer = datasets.load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target)

# Pipelineの構築
パイプラインを使うと前処理から学習器までのチェインを一つのEstimatorインターフェイスとして定義できる。これを使う目的として交差検定やグリッドサーチで正則化・標準化を使う場合が挙げられる。これらはデータを訓練・評価用に分けた後にスケール訓練を訓練データに対して行う必要があるが、cross_val_scoreは値分割後と学習器訓練の間に処理を挟み込むことが出来ない。そこでパイプラインを使う。

パイプラインでは名前とペアにした変換器・学習器をリスト(steps)として渡す事で生成できる。その際、リストの最後に学習器(predictを持つ)を置き、他要素は変換器(transformを持つ)を使う。

In [18]:
param_grid = {
    "svm__C": [0.001, 0.01, 0.1, 1, 10, 100],
    "svm__gamma": [0.001, 0.01, 0.1, 1, 10, 100],
}

pipe = Pipeline([("scaler", MinMaxScaler()), ("svm", SVC())])
grid = GridSearchCV(pipe, param_grid, cv=3).fit(X_train, y_train)

print(classification_report(y_test, grid.predict(X_test)))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97        59
           1       0.98      0.98      0.98        84

   micro avg       0.97      0.97      0.97       143
   macro avg       0.97      0.97      0.97       143
weighted avg       0.97      0.97      0.97       143



make_pipelineを使うとステップの名称をクラス名から自動生成できる。

In [22]:
pipe = make_pipeline(MinMaxScaler(), SVC())
grid = GridSearchCV(pipe, param_grid, cv=3).fit(X_train, y_train)

print(classification_report(y_test, grid.predict(X_test)))

              precision    recall  f1-score   support

           0       0.97      0.97      0.97        59
           1       0.98      0.98      0.98        84

   micro avg       0.97      0.97      0.97       143
   macro avg       0.97      0.97      0.97       143
weighted avg       0.97      0.97      0.97       143



グリッドサーチでは学習器を選ばせることも可能

In [31]:
param_grid = [
    {
        "svm": [LogisticRegression(solver="lbfgs")]
    },
    {
        "svm": [SVC()],
        "svm__C": [0.001, 0.01, 0.1, 1, 10, 100],
        "svm__gamma": [0.001, 0.01, 0.1, 1, 10, 100],
    }
]

pipe = Pipeline([("scaler", MinMaxScaler()), ("svm", SVC())])
grid = GridSearchCV(pipe, param_grid, cv=3).fit(X_train, y_train)

print(grid.best_estimator_)
print(classification_report(y_test, grid.predict(X_test)))

Pipeline(memory=None,
     steps=[('scaler', MinMaxScaler(copy=True, feature_range=(0, 1))), ('svm', SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False))])
              precision    recall  f1-score   support

           0       0.97      0.97      0.97        59
           1       0.98      0.98      0.98        84

   micro avg       0.97      0.97      0.97       143
   macro avg       0.97      0.97      0.97       143
weighted avg       0.97      0.97      0.97       143

